# Round of 16 draw for 2017-2018 season

Nyon - 11 December 2017, 12:00 CET

Draw procedure:

* Two seeding pots will be formed: one consisting of group winners and the other of runners-up.
* No team can play a club from their group or any side from their own association.
* Seeded group winners will be away in the round of 16 first legs and at home in the return matches.

|         **POT Winners**        |      **POT Runners-up**       |
| :----------------------------: | :---------------------------: |
| Manchester United (A, England) | Basel (A, Switzerland)        |
| PSG (B, France)                | Bayer Munchen (B, Germany)    |
| Roma (C, Italy)                | Chelsea (C, England)          |
| Barcelona (D, Spain)           | Juventus (D, Italy)           |
| Liverpool (E, England)         | Sevilla (E, Spain)            |
| Manchester City (F, England)   | Shakhtar Donetsk (F, Ukraine) |
| Besitkas (G, Turkey)           | Porto (G, Portugal)           |
| Tottenham (H, England)         | Real Madrid (H, Spain)        |

Sources:

* [UEFA Champions League round of 16 draw](https://www.uefa.com/uefachampionsleague/season=2018/draws/round=2000882/index.html#/)

## Imports, constants and functions

In [1]:
import itertools
import numpy as np
from IPython.display import display_html

In [2]:
class Team:
    """
    Team representation storing club information regarding:
        - short name
        - association/country to which the club belongs
        - group in the previous stage of the Champions League
    """
    def __init__(self, name, country, group):  # Constructor
        self.name = name
        self.country = country
        self.group = group

    def __repr__(self):  # String representation of instances
        return '{} ({}, {})'.format(self.name, self.group, self.country)

    def __hash__(self):  # Required for list.index working
        return hash((self.name, self.country, self.group))

    def __eq__(self, other):  # Required for list.index working
        try:
            return (self.name, self.country, self.group) == (other.name, other.country, other.group)
        except AttributeError:
            return NotImplemented

In [3]:
def is_valid_draw(fixtures):
    """
    A draw is valid if each game confronts teams belonging to different countries and different groups
    """
    return all([w.group != r.group and w.country != r.country for w, r in fixtures])

def print_html(string):
    """
    Utility function to display HTML in a code cell
    """
    display_html(string, raw=True)

In [4]:
# Winners pot composition
WINNERS = [Team('Manchester United', 'England', 'A'), Team('PSG', 'France', 'B'),
           Team('Roma', 'Italy', 'C'),                Team('Barcelona', 'Spain', 'D'),
           Team('Liverpool', 'England', 'E'),         Team('Manchester City', 'England', 'F'),
           Team('Besitkas', 'Turkey', 'G'),           Team('Tottenham', 'England', 'H')]

# Runners-up pot composition
RUNNERS_UP = [Team('Basel', 'Switzerland', 'A'), Team('Bayer Munchen', 'Germany', 'B'),
              Team('Chelsea', 'England', 'C'),   Team('Juventus', 'Italy', 'D'),
              Team('Sevilla', 'Spain', 'E'),     Team('Shakhtar Donetsk', 'Ukraine', 'F'),
              Team('Porto', 'Portugal', 'G'),    Team('Real Madrid', 'Spain', 'H')]

## Draws calculation

In this case it is feasible to calculate all the possible outcomes of the draw. In a purely random draw, there would be $8! =40320$. Not all of them will be valid draws. There are two constraints about matches: a valid draw has no two teams from the same country and no two teams from the same group in the previous stage of the competition. After removing these draws from the list, there are still $4238$ valid draws. 

In [5]:
draws = [zip(WINNERS, x) for x in itertools.permutations(RUNNERS_UP)]
print("Total number of draws: %d" % len(draws))
valid_draws = filter(lambda x: is_valid_draw(x), draws)
print("Total number of valid draws: %d" % len(valid_draws))

Total number of draws: 40320
Total number of valid draws: 4238


## Probability calculation

As it was explained below, it is possible to compute all the possible draw outcomes. The probability of a game will be the the ratio of favourable outcomes (valid draws containing that game) to the total number of possible outcomes (total number of valid draws). So, no Montecarlo simulation is necessary.

In [6]:
total_events = float(len(valid_draws))  # total number of events
probabilities = np.full((len(WINNERS), len(RUNNERS_UP)), 0,  dtype=np.float32)  # store the probabilities

# Count how many times each pair of teams are matched in a valid draw
for draw in valid_draws:
    for winner, runner_up in draw:
        probabilities[WINNERS.index(winner), RUNNERS_UP.index(runner_up)] += 1
# Probability: the ratio of favourable outcomes to the total number of possible outcomes
probabilities = probabilities/total_events

In [7]:
# HTML output
html = "<table>"
html += "<tr><td>&nbsp;</td><td><b>%s</b></td><td>CHECK</td></tr>" % ("</b></td><td><b>".join([x.name for x in RUNNERS_UP]))
for w_idx in range(len(WINNERS)):
    html += "<tr><td><b>%s</b></td><td>%s</td><td>%.1f</td></tr>" % (WINNERS[w_idx].name, \
                                                                       "</td><td>".join([str(round(x, 3)) \
                                                                          for x in probabilities[w_idx,:]]), \
                                                                     sum(probabilities[w_idx,:]))
html += "<tr><td>CHECK</td><td>%s</td><td>&nbsp;</td></tr>" % ("</td><td>".join([str(round(sum(probabilities[:,x]), 1)) \
                                                                                  for x in range(len(RUNNERS_UP))]))
html += "</table>"
print_html(html)

,Basel,Bayer Munchen,Chelsea,Juventus,Sevilla,Shakhtar Donetsk,Porto,Real Madrid,CHECK
Manchester United,0.0,0.148,0.0,0.183,0.183,0.153,0.148,0.183,1.0
PSG,0.113,0.0,0.281,0.128,0.128,0.113,0.108,0.128,1.0
Roma,0.158,0.153,0.0,0.0,0.189,0.158,0.153,0.189,1.0
Barcelona,0.146,0.135,0.437,0.0,0.0,0.146,0.135,0.0,1.0
Liverpool,0.158,0.153,0.0,0.189,0.0,0.158,0.153,0.189,1.0
Manchester City,0.153,0.148,0.0,0.183,0.183,0.0,0.148,0.183,1.0
Besitkas,0.113,0.108,0.281,0.128,0.128,0.113,0.0,0.128,1.0
Tottenham,0.158,0.153,0.0,0.189,0.189,0.158,0.153,0.0,1.0
CHECK,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,


Some remarks:

* Barcelona vs Chelsea is the most likely fixture
* PSG vs Porto and Besitkas vs Bayer Munchen are the less likely fixtures.